# Schema-based information retrieval from judgements


In [ ]:
# | eval: false
%load_ext autoreload
%autoreload 2

In [ ]:
# | eval: false
from loguru import logger

import mlflow

from juddges.prompts.information_extraction import (
    prepare_information_extraction_chain,
    EXAMPLE_SCHEMA,
)
from juddges.settings import prepare_langchain_cache, prepare_mlflow
from juddges.data.pl_court_api import PolishCourtAPI

In [ ]:
# | eval: false
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [ ]:
# |eval: false
prepare_langchain_cache()
prepare_mlflow()

In [ ]:
# | eval: false
logger.add(level="INFO", sink="../../logs/ir_judgments.log")

1

In [ ]:
LLM_NAME = "gpt-4-0125-preview"
PROMPT_VERSION = "initial-ir"
LANGUAGE = "Polish"

In [ ]:
# | eval: false
mlflow.start_run(run_name=f"{PROMPT_VERSION}-{LLM_NAME}")

<ActiveRun: >

In [ ]:
# | eval: false
api = PolishCourtAPI()

In [ ]:
# | eval: false
judgement = api.get_content(id="155000000001006_II_AKa_000334_2019_Uz_2020-02-06_001")

In [ ]:
# | eval: false
judgement

{'content': '<?xml version=\'1.0\' encoding=\'UTF-8\'?>\n<xPart xPublisherFullName="Anna Turek" xPublisher="anna.turek" xEditorFullName="Wiktoria Dąbrowicz" xEditor="wiktoria.dabrowicz" xVersion="1.0" xLang="PL" xFromPg="1" xToPage="2" xFlag="published" xVolType="15/500000/0001006/AKa" xYear="2019" xVolNmbr="000334" xDocType="Uz" xml:space="preserve">\n  <xName>Wyrok</xName>\n  <xBlock>\n    <xText xALIGNx="left">\n      <xBx>Sygn. akt II AKa 334/19</xBx></xText>\n    <xText/>\n    <xUnit xBold="true" xIsTitle="true" xType="part">\n      <xName>WYROK</xName>\n      <xTitle>W IMIENIU RZECZYPOSPOLITEJ POLSKIEJ</xTitle>\n      <xText xALIGNx="right">Dnia 6 lutego 2020 r.</xText>\n      <xText><xBx>Sąd Apelacyjny we Wrocławiu II Wydział Karny w składzie:</xBx></xText>\n      <xText>Przewodniczący: sędzia SA Cezariusz Baćkowski</xText>\n      <xText>Sędziowie: SA Piotr Kaczmarek (spr.)</xText>\n      <xText>SA Robert Zdych</xText>\n      <xText>Protokolant: Wiktoria Dąbrowicz</xText>\n     

In [ ]:
# | eval: false
print(EXAMPLE_SCHEMA)

verdict_date: date as ISO 8601
verdict: string, text representing verdict of the judgement
verdict_summary: string, short summary of the verdict
verdict_id: string
court: string
parties: string
appeal_against: string
first_trial: boolean
drug_offence: boolean
child_offence: boolean
offence_seriousness: boolean
verdict_tags: List[string]


In [ ]:
# | eval: false
chain = prepare_information_extraction_chain()

In [ ]:
# | eval: false
logged_model = mlflow.langchain.log_model(chain, "langchain_model")

2024/04/12 12:20:42 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp_e0svs4s/model, flavor: langchain). Fall back to return ['langchain==0.1.13', 'pydantic==2.7.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 


In [ ]:
# | eval: false
mlflow.log_param("prompt_version", PROMPT_VERSION)
mlflow.log_param("llm_name", LLM_NAME)
mlflow.log_param("language", LANGUAGE)
mlflow.log_param("schema", EXAMPLE_SCHEMA)

'verdict_date: date as ISO 8601\nverdict: string, text representing verdict of the judgement\nverdict_summary: string, short summary of the verdict\nverdict_id: string\ncourt: string\nparties: string\nappeal_against: string\nfirst_trial: boolean\ndrug_offence: boolean\nchild_offence: boolean\noffence_seriousness: boolean\nverdict_tags: List[string]'

In [ ]:
EXAMPLE_SCHEMA = """
Rodzaj_przestepstwa_i_kwalifikacja_prawna: string
Dodatkowe_okolicznosci: {'recydywa': 'boolean', 'w_zwiazku_z_art_207_kk_znecanie_sie': 'boolean', 'zgwalcenie_maloletniego_zbiorowe_kazirodcze_ze_szczegolnym_okrucienstwem': 'boolean'}
Rodzaj_wyroku: string
Sankcja_karna: {'wyrok_w_zawieszeniu': 'boolean', 'skazujacy_na_kare_pobawienia_wolnosci_od_lat_2_do_lat_3': 'boolean', 'skazujacy_na_kare_pobawienia_wolnosci_od_lat_3': 'boolean', 'kara_25_lat_pobawienia_wolnosci': 'boolean'}
Srodki_kompensacyjne: {'czy_jest': 'boolean', 'jaka_kwota': 'integer'}
Srodki_karne: {'jakie': 'string'}
Okolicznosci_zwiazane_z_opisem_czynu: {'stosowanie_przemocy': 'boolean', 'grozby_bezprawnej': 'boolean', 'podstepu': 'boolean', 'wykorzystanie_bezradnosci': 'boolean', 'znajdowanie_sie_pod_wplywem_alkoholu': 'boolean', 'podanie_GHB_innej_podobnie_dzialajacej_substancji': 'boolean'}
"""

In [ ]:
# | eval: false
retrieved_informations = chain.invoke(
    {"LANGUAGE": LANGUAGE, "TEXT": judgement, "SCHEMA": EXAMPLE_SCHEMA}
)

In [ ]:
# | eval: false
retrieved_informations

{'Rodzaj_przestepstwa_i_kwalifikacja_prawna': 'art. 258 § 1 k.k., art. 55 ust. 3 ustawy z dnia 29 lipca 2005 roku o przeciwdziałaniu narkomanii, art. 12 k.k., art. 65 § 1 k.k., art. 56 ust. 3 ustawy z dnia 29 lipca 2005 roku o przeciwdziałaniu narkomanii, art. 299 § 1 k.k., art. 18 § 3 k.k.',
 'Dodatkowe_okolicznosci': {'recydywa': False,
  'w_zwiazku_z_art_207_kk_znecanie_sie': False,
  'zgwalcenie_maloletniego_zbiorowe_kazirodcze_ze_szczegolnym_okrucienstwem': False},
 'Rodzaj_wyroku': 'zmiana wyroku; uniewinnienie od niektórych przestępstw; utrzymanie w mocy pozostałej części wyroku',
 'Sankcja_karna': {'wyrok_w_zawieszeniu': False,
  'skazujacy_na_kare_pobawienia_wolnosci_od_lat_2_do_lat_3': False,
  'skazujacy_na_kare_pobawienia_wolnosci_od_lat_3': True,
  'kara_25_lat_pobawienia_wolnosci': False},
 'Srodki_kompensacyjne': {'czy_jest': False, 'jaka_kwota': 0},
 'Srodki_karne': {'jakie': ''},
 'Okolicznosci_zwiazane_z_opisem_czynu': {'stosowanie_przemocy': False,
  'grozby_bezprawn

In [ ]:
# | eval: false
from juddges.data.models import get_mongo_collection

judgements_collection = get_mongo_collection("judgements")

In [ ]:
# | eval: false
# search for mongo documents with empty text field, index named `text_text`
judgements = list(
    judgements_collection.find({"$text": {"$search": "przestepstwo seksualne"}}).limit(10)
)

In [ ]:
# | eval: false
# judgements = list(judgements_collection.find(filter={}).limit(100))

In [ ]:
# | eval: false
len(judgements)

10

Estimate price of prompt with judgements text for 100 sample (is USD)


In [ ]:
# | eval: false
from juddges.settings import num_tokens_from_string, LLM_TO_PRICE_INPUT

sum(
    [
        LLM_TO_PRICE_INPUT[LLM_NAME] * num_tokens_from_string(judgement["text"])
        for judgement in judgements
    ]
)

1.53363

In [ ]:
# | eval: false
print(judgements[0]["text"])

WYROK ŁĄCZNY

W IMIENIU

RZECZYPOSPOLITEJ POLSKIEJ

Dnia 07 maja 2014 roku.

Sąd Okręgowy w Poznaniu w III Wydziale Karnym w składzie:

Przewodnicząca: SSO Katarzyna Wolff
Protokolant: Patrycja Rataj
przy udziale Prokuratora Wojskowej Prokuratury Okręgowej w Poznaniu Agnieszka Hildebrandt
po rozpoznaniu dnia 07 maja 2014r

sprawy:

A. W.,synaR.iE.z domuS.,urodzonego (...)wP.

skazanego prawomocnymi wyrokami:

I
Sądu Rejonowego w Poznaniu z dnia 17 czerwca 1999 roku sygn. aktVK 667/99za przestępstwo zart. 279§1kkpopełnione w dniu 24.12.1998r na karę 2 lat pozbawienia wolności z warunkowym zawieszeniem na okres 5 lat próby oraz grzywnę w wysokości 100 stawek dziennych po 20 złotych, po czym postanowieniem Sądu Rejonowego w Poznaniu zarządzono wykonanie kary pozbawienia wolności

II
Sądu Rejonowego w Wałczu z dnia 24.09.2002r sygn. aktIIK 468/02za przestępstwo zart. 209 § 1 kkpopełnione w okresie od 20.03.2001r do lutego 2002r na karę 10 miesięcy ograniczenia wolności polegającej na wykon

In [ ]:
# | eval: false
# save sample judgments
import pandas as pd
from juddges.settings import SAMPLE_DATA_PATH

In [ ]:
# | eval: false
df = pd.DataFrame(judgements)
df.to_csv(SAMPLE_DATA_PATH / "judgements-konfiskata-10-sample.csv")

In [ ]:
# | eval: false
df.head(2)

,_id,signature,date,publicationDate,lastUpdate,courtId,departmentId,type,excerpt,content,...,legalBases,publisher,recorder,references,reviser,themePhrases,num_pages,text,vol_number,vol_type
0,153510000001506_III_K_000042_2014_Uz_2014-05-0...,III K 42/14,2014-05-07 02:00:00.0 CEST,2014-05-30 20:15:03.0 CEST,2014-06-17 00:57:36.0 CEST,15351000,1506,SENTENCE,WYROK ŁĄCZNY W IMIENIU RZECZYPOSPOLITEJ POLSKI...,<?xml version='1.0' encoding='UTF-8'?>\n<xPart...,...,[art. 85 i art. 86 § 1 i 2 kk],Justyna Grzegorek,Patrycja Rataj,[Ustawa z dnia 23 czerwca 1973 r. o opłatach w...,Agnieszka Sikorska,[Wyrok Łączny],4,WYROK ŁĄCZNY\n\nW IMIENIU\n\nRZECZYPOSPOLITEJ ...,42,15/351000/0001506/K
1,150515000003506_VII_Ka_000726_2013_Uz_2013-09-...,VII Ka 726/13,2013-09-04 02:00:00.0 CEST,2014-02-05 19:15:04.0 CET,2014-09-07 10:18:28.0 CEST,15051500,3506,"SENTENCE, REASON",Sygn. akt VII Ka 726/13 WYROK W IMIENIU RZECZY...,<?xml version='1.0' encoding='UTF-8'?>\n<xPart...,...,[Art.437§1 Kpk],Wioletta Suraj,sekr.sądowy Elżbieta Łotowska,[Ustawa z dnia 26 maja 1982 r. - Prawo o adwok...,Rafał Banaszewski,[Postępowanie Odwoławcze],18,Sygn. akt VII Ka 726/13\n\nWYROK\nW IMIENIU RZ...,726,15/051500/0003506/Ka


In [ ]:
# | eval: false
judgements_retrieved_informations = chain.batch(
    [
        {"LANGUAGE": LANGUAGE, "TEXT": judgement["text"], "SCHEMA": EXAMPLE_SCHEMA}
        for judgement in judgements
    ]
)

In [ ]:
# | eval: false

# add list of retrieved informations to each judgement to the data frame df but treat each key in dict as a new column
from tqdm import tqdm

for key in tqdm(judgements_retrieved_informations[0].keys(), desc="Adding columns to DataFrame"):
    df[key] = [judgement[key] for judgement in judgements_retrieved_informations]

Adding columns to DataFrame: 100%|██████████| 7/7 [00:00<00:00, 2339.83it/s]


In [ ]:
# | eval: false
df.head(2)

,_id,signature,date,publicationDate,lastUpdate,courtId,departmentId,type,excerpt,content,...,text,vol_number,vol_type,Rodzaj_przestepstwa_i_kwalifikacja_prawna,Dodatkowe_okolicznosci,Rodzaj_wyroku,Sankcja_karna,Srodki_kompensacyjne,Srodki_karne,Okolicznosci_zwiazane_z_opisem_czynu
0,153510000001506_III_K_000042_2014_Uz_2014-05-0...,III K 42/14,2014-05-07 02:00:00.0 CEST,2014-05-30 20:15:03.0 CEST,2014-06-17 00:57:36.0 CEST,15351000,1506,SENTENCE,WYROK ŁĄCZNY W IMIENIU RZECZYPOSPOLITEJ POLSKI...,<?xml version='1.0' encoding='UTF-8'?>\n<xPart...,...,WYROK ŁĄCZNY\n\nW IMIENIU\n\nRZECZYPOSPOLITEJ ...,42,15/351000/0001506/K,,"{'recydywa': True, 'w_zwiazku_z_art_207_kk_zne...",wyrok łączny,"{'wyrok_w_zawieszeniu': False, 'skazujacy_na_k...","{'czy_jest': False, 'jaka_kwota': 0}",{'jakie': ''},"{'stosowanie_przemocy': False, 'grozby_bezpraw..."
1,150515000003506_VII_Ka_000726_2013_Uz_2013-09-...,VII Ka 726/13,2013-09-04 02:00:00.0 CEST,2014-02-05 19:15:04.0 CET,2014-09-07 10:18:28.0 CEST,15051500,3506,"SENTENCE, REASON",Sygn. akt VII Ka 726/13 WYROK W IMIENIU RZECZY...,<?xml version='1.0' encoding='UTF-8'?>\n<xPart...,...,Sygn. akt VII Ka 726/13\n\nWYROK\nW IMIENIU RZ...,726,15/051500/0003506/Ka,kradzież z włamaniem,"{'recydywa': True, 'w_zwiazku_z_art_207_kk_zne...",utrzymanie wyroku w mocy,"{'wyrok_w_zawieszeniu': False, 'skazujacy_na_k...","{'czy_jest': True, 'jaka_kwota': 3386243}",{'jakie': ''},"{'stosowanie_przemocy': False, 'grozby_bezpraw..."


In [ ]:
# | eval: false
df.to_csv(SAMPLE_DATA_PATH / "judgements-10-konfiskata-sample-with-retrieved-informations.csv")